##Exercício de manipulação de dados coletados do Mercado Livre

In [24]:
import pandas as pd
import re #importar regex
import numpy as np #para NaN

In [3]:
#abrindo o DataFrame de dados:
df_ml = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/exercicios_aulas/Coleta de dados/dados_mercado_livre.csv')
df_ml.head()

,zonas,areas_quartos,enderecos,precos
0,sul,175 m² construídos4 quartos,"Rua Deputado João Sussumu Hirata, 38, Vila And...",4.5
1,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4.4
2,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4.4
3,sul,120 m² construídos3 quartos,"Jardim Prudência, São Paulo Zona Sul, São Paulo",4.4
4,sul,66 m² construídos2 quartos,"Rua Celso Ramos, Vila Andrade, São Paulo Zona ...",2.9


In [8]:
df_ml = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/exercicios_aulas/Coleta de dados/dados_mercado_livre.csv', thousands='.')
df_ml.head()

,zonas,areas_quartos,enderecos,precos
0,sul,175 m² construídos4 quartos,"Rua Deputado João Sussumu Hirata, 38, Vila And...",4500
1,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400
2,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400
3,sul,120 m² construídos3 quartos,"Jardim Prudência, São Paulo Zona Sul, São Paulo",4400
4,sul,66 m² construídos2 quartos,"Rua Celso Ramos, Vila Andrade, São Paulo Zona ...",2900


In [5]:
df_ml.info() #não temos dados nulos (considerando area e quartos juntos, ao separar, não terá quartos em todas as linhas)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   zonas          960 non-null    object
 1   areas_quartos  960 non-null    object
 2   enderecos      960 non-null    object
 3   precos         960 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 30.1+ KB


###Precisamos separar áreas e quartos em colunas diferentes:

In [12]:
#testes para regex:
texto = '175 m² construídos4 quartos'
padrao = '(\d+)'

In [13]:
re.findall(padrao, texto)

['175', '4']

In [14]:
re.findall(padrao, texto)[0]

'175'

In [15]:
re.findall(padrao, texto)[1]

'4'

In [21]:
#acrescentar m² ao regex me traz de volta apenas o m²
texto1 = '175 m² construídos4 quartos'
padrao1 = '(\d+) m²'

In [22]:
#me traz o m² dentro de conchetes
re.findall(padrao1, texto1)

['175']

In [23]:
#precisa acrescentar a posição 0 para sair do conchete
re.findall(padrao1, texto1)[0]

'175'

In [18]:
#acrescentar quarto ao regex me traz de volta apenas o número de quartos
texto = '175 m² construídos4 quartos'
padrao = '(\d+) quarto'

In [19]:
#me traz o número de quartos dentro de conchetes
re.findall(padrao, texto)

['4']

In [20]:
#precisa acrescentar a posição 0 para sair do conchete
re.findall(padrao, texto)[0]

'4'

###Extração de variáveis com regex:

In [25]:
def extrair_area(entrada):
  if 'm²' in entrada:
    area = re.findall('(\d+) m²', entrada)[0]
  else:
    area = np.nan
  return area

In [26]:
def extrair_quarto(entrada):
  if 'quartos' in entrada:
    n_quartos = re.findall('(\d+) quarto', entrada)[0]
  else:
    n_quartos = np.nan
  return n_quartos

In [29]:
df_ml['areas'] = df_ml['areas_quartos'].apply(extrair_area)
df_ml['areas']

0      175
1      150
2      150
3      120
4       66
      ... 
955     42
956     50
957     76
958     43
959     30
Name: areas, Length: 960, dtype: object

In [41]:
df_ml['quartos'] = df_ml['areas_quartos'].apply(extrair_quarto)
df_ml['quartos']

0        4
1        3
2        3
3        3
4        2
      ... 
955      2
956      2
957      3
958    NaN
959    NaN
Name: quartos, Length: 960, dtype: object

###Separando areas_quartos por split (como em aula)

In [31]:
def extrair_area_split(entrada):
  if 'm²' in entrada:
    area = entrada.split('m²')[0]
  else:
    area = np.nan
  return area

In [34]:
df_ml['areas'] = df_ml['areas_quartos'].apply(extrair_area_split)
df_ml['areas']

0      175 
1      150 
2      150 
3      120 
4       66 
       ... 
955     42 
956     50 
957     76 
958     43 
959     30 
Name: areas, Length: 960, dtype: object

###Transformando dados em tipo integer

In [43]:
df_ml['precos'] = df_ml['precos'].astype(int)
df_ml['areas'] = df_ml['areas'].astype(int)
df_ml['quartos'] = df_ml['quartos'].astype(float) 
#Não me deixa colocar int em quartos porque tem NaN

In [45]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos,areas,quartos
0,sul,175 m² construídos4 quartos,"Rua Deputado João Sussumu Hirata, 38, Vila And...",4500,175,4.0
1,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
2,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
3,sul,120 m² construídos3 quartos,"Jardim Prudência, São Paulo Zona Sul, São Paulo",4400,120,3.0
4,sul,66 m² construídos2 quartos,"Rua Celso Ramos, Vila Andrade, São Paulo Zona ...",2900,66,2.0


In [ ]:
#não precisamos mais da coluna area-quartos, então:

###Dropando coluna - salvando novo arquivo CSV

In [47]:
df_ml.drop(columns='areas_quartos')

,zonas,enderecos,precos,areas,quartos
0,sul,"Rua Deputado João Sussumu Hirata, 38, Vila And...",4500,175,4.0
1,sul,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
2,sul,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
3,sul,"Jardim Prudência, São Paulo Zona Sul, São Paulo",4400,120,3.0
4,sul,"Rua Celso Ramos, Vila Andrade, São Paulo Zona ...",2900,66,2.0
...,...,...,...,...,...
955,norte,"Rua Coronel Euclídes Machado, Jardim Peri, São...",1300,42,2.0
956,norte,"Rua Soldado Sebastião Garcia, Parque Novo Mund...",1500,50,2.0
957,norte,"Rua Felício Tarabai, Barro Branco, São Paulo Z...",1300,76,3.0
958,norte,"Rua Igarité, Vila Nova Carolina, São Paulo Zon...",1050,43,NaN


In [ ]:
#Ou:
df_ml.drop('areas_quartos', axis=1)
#aqui eu preciso especificar em axis o número de posição da coluna a ser dropada, no caso, 1 (0 era zonas)

In [48]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos,areas,quartos
0,sul,175 m² construídos4 quartos,"Rua Deputado João Sussumu Hirata, 38, Vila And...",4500,175,4.0
1,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
2,sul,150 m² construídos3 quartos,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
3,sul,120 m² construídos3 quartos,"Jardim Prudência, São Paulo Zona Sul, São Paulo",4400,120,3.0
4,sul,66 m² construídos2 quartos,"Rua Celso Ramos, Vila Andrade, São Paulo Zona ...",2900,66,2.0


In [ ]:
'''
só dropar a coluna não a apaga indefinidamente:
ou colocamos um inplace=True junto à dropagem, que apaga sem volta
ou salvamos o DataFrame com drop em outra variável

In [49]:
df_final = df_ml.drop(columns='areas_quartos')

In [50]:
df_final.head()

,zonas,enderecos,precos,areas,quartos
0,sul,"Rua Deputado João Sussumu Hirata, 38, Vila And...",4500,175,4.0
1,sul,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
2,sul,"Rua Manuel Da Nóbrega, 533, Paraíso, São Paulo...",4400,150,3.0
3,sul,"Jardim Prudência, São Paulo Zona Sul, São Paulo",4400,120,3.0
4,sul,"Rua Celso Ramos, Vila Andrade, São Paulo Zona ...",2900,66,2.0


In [51]:
df_final.info() #apenas para conferir - bastante dados faltantes em quartos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   zonas      960 non-null    object 
 1   enderecos  960 non-null    object 
 2   precos     960 non-null    int64  
 3   areas      960 non-null    int64  
 4   quartos    690 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 37.6+ KB


In [53]:
#salvando em novo arquivo csv:
df_final.to_csv('Manipulação de dados ML.csv')